In [12]:
# all dirs
import os
dirs = ['/'.join(['data', tmp]) for tmp in os.listdir('data/') if os.path.isdir('/'.join(['data', tmp]))]
dirs

['data/NG_Ka_Long_Angus_SHI_Yu_Qi_Thailand_Masters_2020_SemiFinals',
 'data/Ng_Ka_Long_Angus_Kidambi_Srikanth_HSBC_BWF_WORLD_TOUR_FINALS_2020_QuarterFinals',
 'data/Viktor_Axelsen_Liew_Daren_TOYOTA_THAILAND_OPEN_2021_QuarterFinals',
 'data/Anthony_Sinisuka_GINTING_Anders_ANTONSEN_Indonesia_Masters_2020_Final',
 'data/Kento_MOMOTA_CHOU_Tien_Chen_Fuzhou_Open_2018_Finals',
 'data/Kento_MOMOTA_CHOU_Tien_Chen_Malaysia_Open_2018_QuarterFinals',
 'data/Anders_ANTONSEN_Jonatan_CHRISTIE Indonesia_Masters_2020_QuarterFinals',
 'data/CHOU_Tien_Chen_NG_Ka_Long_Angus_Sudirman_Cup_2019_Group_Stage',
 'data/CHOU_Tien_Chen_Jonatan_CHRISTIE_Indonesia_Open_2019_Quarter-finals',
 'data/Hans-Kristian_Solberg_Vittinghus_Anders_Antonsen_TOYOTA_THAILAND_OPEN_2021_SemiFinals',
 'data/Viktor_AXELSEN_NG_Ka_Long_Angus_Malaysia_Masters_2020_SemiFinals',
 'data/Carolina_Marin_Neslihan_Yigit_TOYOTA_THAILAND_OPEN_2021_QuarterFinals',
 'data/Carolina_Marin_Supanida_Katethong_YONEX_Thailand_Open_2021_QuarterFinals',
 

In [40]:
# each file process
import pandas as pd
import numpy as np
def extract_info(file
                       ='data/An_Se_Young_Pornpawee_Chochuwong_TOYOTA_THAILAND_OPEN_2021_QuarterFinals/set1.csv',
                 contents=['rally', 'player', 'hit_area', 'hit_x', 'hit_y', 'landing_area', 'landing_x', 'landing_y', 'type']):
    df = pd.read_csv(file)
    df = df[contents]
    return df

def extract_metainfo(matchname='An_Se_Young_Pornpawee_Chochuwong_TOYOTA_THAILAND_OPEN_2021_QuarterFinals'):
    homograph = pd.read_csv('data/homography.csv')
    matchgraph = homograph[homograph['video'] == matchname]
    upleft, upright, downleft, downright = (
        matchgraph['upleft_x'].values[0], matchgraph['upleft_y'].values[0]), (
        matchgraph['upright_x'].values[0], matchgraph['upright_y'].values[0]), (
        matchgraph['downleft_x'].values[0], matchgraph['downleft_y'].values[0]), (
        matchgraph['downright_x'].values[0], matchgraph['downright_y'].values[0])
    return upleft[0], upleft[1], upright[0], upright[1], downleft[0], downleft[1], downright[0], downright[1]


upleft_x, upleft_y, upright_x, upright_y, downleft_x, downleft_y, downright_x, downright_y = extract_metainfo()

LOWER_LEFT = (downleft_x, downleft_y)
LOWER_RIGHT = (downright_x, downleft_y)
MIDDLE_LEFT = (int((upleft_x+downleft_x)/2), int((upleft_y+downleft_y)/2))
MIDDLE_RIGHT = (int((upright_x+downright_x)/2), int((upright_y+downright_y)/2))
UPPER_LEFT = (upleft_x, upleft_y)
UPPER_RIGHT = (upright_x, upright_y)


def lefter(x, y, p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    if y1 == y2:
        return False
    return (x-x1) - (y-y1)*(x2-x1)/(y2-y1) < 0


def upper(x, y, p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    if x1 == x2:
        return False
    return (y-y1) - (x-x1)/(x2-x1)*(y2-y1) < 0


def pixel2gridcoord(x, y, n=3):
    upper_horizontal_lines = [((MIDDLE_LEFT[0]*i/n+UPPER_LEFT[0]*(n-i)/n,
                                MIDDLE_LEFT[1]*i/n+UPPER_LEFT[1]*(n-i)/n),
                                (MIDDLE_RIGHT[0]*i/n+UPPER_RIGHT[0]*(n-i)/n,
                                MIDDLE_RIGHT[1]*i/n+UPPER_RIGHT[1]*(n-i)/n)) for i in range(1, n)]
    lower_horizontal_lines = [((LOWER_LEFT[0]*i/n+MIDDLE_LEFT[0]*(n-i)/n,
                                LOWER_LEFT[1]*i/n+MIDDLE_LEFT[1]*(n-i)/n),
                                (LOWER_RIGHT[0]*i/n+MIDDLE_RIGHT[0]*(n-i)/n,
                                LOWER_RIGHT[1]*i/n+MIDDLE_RIGHT[1]*(n-i)/n)) for i in range(1, n)]
    vertical_lines = [((LOWER_LEFT[0]*i/n+LOWER_RIGHT[0]*(n-i)/n, LOWER_LEFT[1]),
                       (UPPER_LEFT[0]*i/n+UPPER_RIGHT[0]*(n-i)/n, UPPER_LEFT[1])) for i in range(1, n).__reversed__()]
    # vertical_lines = [((LOWER_LEFT[0]*i/n+LOWER_RIGHT[0]*(n-i)/n, LOWER_LEFT[1]),
    #                     (UPPER_LEFT[0]*i/n+UPPER_RIGHT[0]*(n-i)/n, UPPER_LEFT[1])) for i in range(0, n+1).__reversed__()]
    # cal block_x
    block_x = 0
    try:
        while not lefter(x, y, vertical_lines[block_x][0], vertical_lines[block_x][1]):
            block_x += 1
    except:
        pass
    # cal block_y
    block_y = 0
    try:
        while not upper(x, y, upper_horizontal_lines[block_y][0], upper_horizontal_lines[block_y][1]):
            block_y += 1
    except:
        pass
    if not upper(x, y, MIDDLE_LEFT, MIDDLE_RIGHT):
        block_y += 1
    y_base = block_y
    try:
        while not upper(x, y, lower_horizontal_lines[block_y-y_base][0], lower_horizontal_lines[block_y-y_base][1]):
            block_y += 1
    except:
        pass
    return block_x, block_y


def space_input(df, n=10):
    ret = []
    for coords in df[['hit_x','hit_y', 'landing_x', 'landing_y']].values:
        hit_x, hit_y, landing_x, landing_y = coords
        if pd.isna(coords).sum() != 0:
            ret.append(np.zeros((2*n, n)))
            continue
        tmp = np.zeros((2*n,n))
        x,y = pixel2gridcoord(hit_x,hit_y,n)
        tmp[y, x] = 1
        x,y = pixel2gridcoord(landing_x, landing_y,n)
        tmp[y, x] = 1
        ret.append(tmp)
    return np.array(ret)

def type_input(df):
    one_hot = pd.get_dummies(df['type'])
    return one_hot

def label_output(df):
    # landing or not
    
    return

df = extract_info()
# print(df)
type_onehot = type_input(df)
space_onehot = space_input(df)


In [38]:
# nn model


array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0.]])

In [ ]:
# train model
